In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from scipy.integrate import odeint

In [26]:
# Physical constant
e = 1.6e-19 #[C] electron charge 
M_i = 32*1.6605e-27 #[kg] ion mass
m = 1.44e-30 #[kg] electron mass
k = 1.38e-23 #[m2 kg /s2 K] Boltzmann constant

# Experimental constant
p = 10/7.5 #[mTorr to Pa]
R = 3e-2 #[m] plasma half-width
L = 33e-2 #[m] chamber length
Tg = 300 #[K] gas temperature
R_0 = 2.5e-2 #[m] chosen normalization length
B_0 = 0.03 #[T] magnetic field
gamma = 20 # electron-ion temperature ratio Te/Ti
Te = 3
Ti = Te/gamma
n_g = p/k/Tg #[m-3] neutral gas density
n_e0 = n_g * 2.75e-4 #experimental condition


In [30]:
class fluid_model():
    
    def rate_constant(self, Te, n_g):
        K_iz =  2.34e-15*Te**1.03*np.exp(-12.29/Te) #[m3/s]
        K_att = 1.07e-15*Te**-1.391*np.exp(-6.26/Te) #[m3/s]
        K_rec = 5.2e-14*(0.026/Ti)**0.44 #[m3/s]
        nu_i = 3.95e-16*n_g #[s-1]
        nu_e = 4.7e-14*Te**0.5*n_g #[s-1]
        nu_n = 3.95e-16*n_g #[s-1]
        
        return K_iz, K_att, K_rec, nu_i, nu_e, nu_n
    
    def normalize(self, n_e0, Te):
        K_iz, K_att, K_rec, nu_i, nu_e, nu_n = self.rate_constant(Te,n_g)
        
        w_ce = e*B_0/m
        Di = e*Ti/(M_i*nu_i)
        nu_L = 2*Di/(R*L)*(1+Te/Ti)**0.5
        
        self.X = x/R_0
        self.I = n_i/n_e0 
        self.G = n_g/n_e0
        self.E = n_e/n_e0 #미지수
        self.N = n_n/n_e0 #미지수
        self.Cs = (e*Te/M_i) 
        self.U = v_i/self.Cs #미지수
        self.V = v_e/self.Cs #미지수
        self.W = v_n/self.Cs #미지수
        self.Phi = -phi/Te #미지수
        self.A_i = n_e0*R_0*K_iz/self.Cs
        self.A_a = n_e0*R_0*K_att/self.Cs
        self.B_i = n_e0*R_0*K_rec/self.Cs
        self.C_i = R_0*nu_i/self.Cs
        self.C_e = R_0*nu_e/self.Cs
        self.C_n = R_0*nu_n/self.Cs
        self.Omega = R_0*w_ce/self.Cs 
        self.Epsilon_i = gamma #Te/Ti는 설정값으로줌
        self.Epsilon_n = gamma #Te/Tn, Ti=Tn
        self.Zeta = 1 #M_i/M_n, Mi=Mn
        self.D = R_0*nu_L/self.Cs
        self.Del = (self.A_i+self.A_a)*G+self.C_e
        self.Delta = self.Del*(m/M_i)*(1+self.Omega**2/self.Del**2)
        self.I = self.E+self.N
        
    def matrix_calculation(self):
        
        self.normalize(n_e0, Te)
        M = np.array([[self.V, 0, 0, self.E, 0, 0],
                      [self.U, self.U, self.I, 0, 0, 0],
                      [0, self.W, 0, 0, self.N, 0],
                      [1, 0, 0, 0, 0, self.E],
                      [1/self.Epsilon_i+self.U**2, 1/self.Epsilon_i+self.U**2, 2*self.I*self.N, 0, 0, -self.I],
                      [0, self.Zeta/self.Epsilon_n+self.W**2, 0, 0, 2*self.N*self.W, self.Zeta*self.N]])
        inv_M = np.linalg.inv(M)
        RHS = np.array([[(self.A_i-self.A_a)*self.G*self.E-self.D*self.I],
                        [self.A_i*self.G*self.E-self.B_i*self.I*self.N-self.D*self.I],
                        [self.A_a*self.G*self.E-self.B_i*self.I*self.N],
                        [-self.Delta*self.E*self.V],
                        [-self.C_i*self.I*self.U],
                        [-self.C_n*self.N*self.W]])
        x = np.transpose(np.dot(inv_M,RHS))[0]
        return x
    
    def integration(self):
        x0 = [0.1, 0.1, 0.5, 0.5, 0.5, 1] #E, N, U, V, W, Phi
        self.x = np.arange(0,1,1e-3)
        ans = odeint(self.matrix_calculation, x0, self.x, rtol=10**-3, mxstep=10**6)
        self.sol_E = ans[0]
        self.sol_N = ans[1]
        self.sol_I = ans[0]+ans[1]
    
    def visualize(self):
        plt.subplot(421)
        plt.figure(figsize=(16,16))
        plt.plot(self.x, self.sol_E)
        plt.xlabel('Normalized x')
        plt.ylabel('Normalized density(E)')
        plt.grid(True)
        
        plt.subplot(422)
        plt.plot(self.x, self.sol_N)
        plt.xlabel('Normalized x')
        plt.ylabel('Normalized density(N)')
        plt.grid(True)
        plt.subplots_adjust(hspace = 0.5)
        
        plt.show()

In [31]:
test_model = fluid_model()
test_model.integration()
test_model.visualize()

TypeError: matrix_calculation() takes 1 positional argument but 3 were given

In [ ]:
    def visualization1(self,i):
        plt.figure(figsize=(16,16))
        Power_list = []
        for i in self.t:
            Power_list.append(self.power(i)/6.241509e18)
    
        plt.subplot(421)
        plt.plot(self.t*1e6,self.T)
        plt.xlabel('Time (us)')
        plt.ylabel('Temperature (eV)')
        #plt.xlim(440,560)
        #plt.ylim(0,10)
        plt.title('Electron Temperature')
        plt.grid(True)

        plt.subplot(422)
        plt.plot(self.t*1e6,self.ne,'brown')
        plt.yscale('log')
        plt.xlabel('Time (us)')
        plt.ylabel('Density (cm-3)')
        plt.title('Electron Density')
        plt.grid(True)
        plt.subplots_adjust(hspace = 0.5)
        plt.savefig(path + str(i) + 'vis1.png')
        plt.show()

In [ ]:
    def calculation(self):
        x0 = [1.5,1e11,1e11,1e11,1e11] #Te, H, H+, H2+, H3+
        self.t = np.linspace(0, self.period, int(self.period/self.time_resolution))
        args = (self.power,)
        ans1 = odeint(self.electron_balance_eqn, x0, self.t, args, rtol=10**-3, mxstep=10**6)
        self.T = ans1[:,0]
        self.H = ans1[:,1]
        self.Hp = ans1[:,2]
        self.H2p = ans1[:,3]
        self.H3p = ans1[:,4]
        self.ne = self.Hp + self.H2p + self.H3p
        self.H2 = self.ng-(0.5*(self.H+self.Hp)+self.H2p+1.5*self.H3p)

In [52]:
M = np.array([[1,2]
              ,[3,4]])
print(M)

[[1 2]
 [3 4]]


In [69]:
M = np.array([[1,2,3]])
print(M.shape)
N = np.transpose(M)
print(N.shape)

Z = np.array([[1],[2],[3]])
a,b,c = np.transpose(Z)[0]

(1, 3)
(3, 1)


In [53]:
print(np.linalg.inv(M))

[[-2.   1. ]
 [ 1.5 -0.5]]


In [77]:
odeint?

Signature:
odeint(
    func,
    y0,
    t,
    args=(),
    Dfun=None,
    col_deriv=0,
    full_output=0,
    ml=None,
    mu=None,
    rtol=None,
    atol=None,
    tcrit=None,
    h0=0.0,
    hmax=0.0,
    hmin=0.0,
    ixpr=0,
    mxstep=0,
    mxhnil=0,
    mxordn=12,
    mxords=5,
    printmessg=0,
    tfirst=False,
)
Docstring:
Integrate a system of ordinary differential equations.

.. note:: For new code, use `scipy.integrate.solve_ivp` to solve a
          differential equation.

Solve a system of ordinary differential equations using lsoda from the
FORTRAN library odepack.

Solves the initial value problem for stiff or non-stiff systems
of first order ode-s::

    dy/dt = func(y, t, ...)  [or func(t, y, ...)]

where y can be a vector.

.. note:: By default, the required order of the first two arguments of
          `func` are in the opposite order of the arguments in the system
          definition function used by the `scipy.integrate.ode` class and
          the function `